# Required Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import random
from openpyxl import load_workbook


# Required Functions 

In [6]:
%run Optimization_RTK_Functions.ipynb

# Input

In [ ]:
file_path = './CCW_event_9.xlsx'                # Sets the file path for the Excel data file.
data = pd.read_excel(file_path, skiprows=0)     # Reads the data from the Excel file into a DataFrame.

rainfall = data.iloc[:,2].dropna().tolist()     # Gets data from the 3rd column (index 2) for rainfall, drops empty rows, and converts to a list.
obs_rdii = data.iloc[:,1].tolist()              # Gets data from the 2nd column (index 1) for observed RDII flow and converts to a list.

delta_t = 600                                   # Sets the time step duration to 600 seconds (which is a 10-minute interval).
area_acres = 491.153                            # Defines the catchment area in acres.

total_rdii_period = (len(obs_rdii) - len(rainfall)) * delta_t # Calculates the decay period after rainfall stops.

### T and K Range 

In [13]:
# Allowed values for T (in seconds, 10-minute steps)
allowed_T1_values = [x * 60 for x in range(10, 121, 10)]  # 10 min to 2 hrs
allowed_T2_values = [x * 60 for x in range(20, 241, 10)]  # 20 min to 4 hrs
allowed_T3_values = [x * 60 for x in range(30, 421, 10)]  # 30 min to 7 hrs

# Allowed values for K (0.001 steps)
allowed_K1_values = [round(x, 3) for x in np.arange(1, 2.01, 0.001)]  # 1to 2
allowed_K2_values = [round(x, 3) for x in np.arange(2, 3.01, 0.001)]  # 2 to 3
allowed_K3_values = [round(x, 3) for x in np.arange(3.0, 7.01, 0.001)]  # 3 to 7


# Differential Evolution Optimization

## Objective Function defination

In [23]:
def map_index_to_value(index, allowed_values):
    index = int(round(index))  # Ensure the index is an integer
    index = max(0, min(index, len(allowed_values) - 1))  # Clip to valid range
    return allowed_values[index]

In [4]:
def objective_function(params_flat, obs_rdii):
    # Unpack parameters
    R1, T1_idx, K1_idx, R2, T2_idx, K2_idx, R3, T3_idx, K3_idx = params_flat

    # **Strict Non-Negative Constraint for R Values**
    if R1 < 0 or R2 < 0 or R3 < 0:
        print("Invalid R values detected (negative), skipping iteration.")
        return float('inf')

    # Map indices to actual values
    T1 = map_index_to_value(T1_idx, allowed_T1_values)
    T2 = map_index_to_value(T2_idx, allowed_T2_values)
    T3 = map_index_to_value(T3_idx, allowed_T3_values)

    K1 = map_index_to_value(K1_idx, allowed_K1_values)
    K2 = map_index_to_value(K2_idx, allowed_K2_values)
    K3 = map_index_to_value(K3_idx, allowed_K3_values)

    # Parameters for RDII calculation
    params = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]

    # Calculate simulated RDII
    sim_rdii = RDII_calculation(params, delta_t, rainfall, area_acres)

  
    sim_rdii = np.array(sim_rdii)

    # Check for invalid outputs
    if sim_rdii.size == 0 or np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
        print("Invalid sim_rdii generated, returning large penalty.")
        return float('inf')

    # Pad the shorter array with zeros to match the length of the longer one
    max_length = max(len(obs_rdii), len(sim_rdii))
    obs_rdii_padded = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), mode='constant')
    sim_rdii_padded = np.pad(sim_rdii, (0, max_length - len(sim_rdii)), mode='constant')


    
    # Calculate fitness value
    fitness_value = fitness(
        obs_rdii_padded,
        sim_rdii_padded,
        delta_t,
        weight_rmse=0.25,
        weight_r2=0.25,
        weight_pbias=0.25,
        weight_nse=0.25
    )

    penalty = 0
    Ro = R_calc(rainfall, obs_rdii, delta_t, area_acres)
    
    # Enforce equality: R1 + R2 + R3 = Ro
    r_sum = R1 + R2 + R3
    penalty += 1e6 * (r_sum - Ro)**2  # quadratic penalty would be (r_sum - Ro)**2 if preferred
    
    # Temporal ordering constraints
    if not (T1 < T2 < T3):
        penalty += 1000
    if not (T1 + T1 * K1 < T2 + T2 * K2 < T3 + T3 * K3 <= total_rdii_period):
        penalty += 1000
    # penalty = 0    


    return fitness_value + penalty


In [27]:
bounds = [
    (0.0, 1.0),  # R1
    (0.0, len(allowed_T1_values) - 1),  # T1_idx
    (0.0, len(allowed_K1_values) - 1),  # K1_idx
    (0.0, 1.0),  # R2
    (0.0, len(allowed_T2_values) - 1),  # T2_idx
    (0.0, len(allowed_K2_values) - 1),  # K2_idx
    (0.0, 1.0),  # R3
    (0.0, len(allowed_T3_values) - 1),  # T3_idx
    (0.0, len(allowed_K3_values) - 1)   # K3_idx
]


## Multiple Run

In [ ]:
# Define the number of runs
num_runs = 20

# Initialize a list to store the results
all_results = []
fitness_values = []

# Run differential evolution multiple times
for run in range(num_runs):
    print(f"Running DE: Run {run + 1}/{num_runs}")
    result = differential_evolution(
        objective_function,
        bounds,
        args=(obs_rdii,),  # Pass obs_rdii as an argument
        strategy='best1bin',
        maxiter=300,
        popsize=60,
        tol=1e-6,
        mutation=(0.5, 1.0),
        recombination=0.70
    )
    
    # Extract the best parameters
    best_parameters = result.x
    R1, T1_idx, K1_idx, R2, T2_idx, K2_idx, R3, T3_idx, K3_idx = best_parameters
    
    # Map indices back to actual values
    T1 = map_index_to_value(T1_idx, allowed_T1_values)
    T2 = map_index_to_value(T2_idx, allowed_T2_values)
    T3 = map_index_to_value(T3_idx, allowed_T3_values)
    
    K1 = map_index_to_value(K1_idx, allowed_K1_values)
    K2 = map_index_to_value(K2_idx, allowed_K2_values)
    K3 = map_index_to_value(K3_idx, allowed_K3_values)
    
    # Final parameter set
    best_params_actual = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]
    
    # Store the results
    all_results.append(best_params_actual)
    fitness_values.append(result.fun)

print("Completed all DE runs.")

In [ ]:
all_params = all_results 
# Combine results and fitness values into a DataFrame
columns = ['R1', 'T1', 'K1', 'R2', 'T2', 'K2', 'R3', 'T3', 'K3', 'Fitness']
results_df = pd.DataFrame([
    [params[0][0], params[0][1], params[0][2],
     params[1][0], params[1][1], params[1][2],
     params[2][0], params[2][1], params[2][2], fitness]
    for params, fitness in zip(all_results, fitness_values)
], columns=columns)

# Calculate statistics
mean_values = results_df.mean()
std_values = results_df.std()

# Display results
print("\nResults from all runs:")
print(results_df)

print("Mean values of parameters:")
print(mean_values)



## Saving RTK parameters

In [ ]:
# Define the Excel file name
excel_filename = "RTK_Parameters_all_algorithms_Ro_constraint_E9_CCW.xlsx"
sheet_name = "DE"

# Check if "Run" column exists, and add it only if not present
if "Run" not in results_df.columns:
    results_df.insert(0, "Run", range(1, len(results_df) + 1))

# Select the desired columns
export_columns = ["Run", "R1", "T1", "K1", "R2", "T2", "K2", "R3", "T3", "K3"]

# Check if the Excel file exists
try:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a") as writer:
        results_df[export_columns].to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file does not exist, create a new one
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="w") as writer:
        results_df[export_columns].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Results successfully saved to {excel_filename} in sheet {sheet_name}.")

In [ ]:
plot=RDII_combined_plot(all_params, delta_t, rainfall, area_acres, obs_rdii)

## Saving Metrices

In [ ]:
criteria= calculate_criteria_multiple_runs(all_results, obs_rdii, delta_t, rainfall, area_acres)

In [ ]:
# Define the Excel file and sheet name
excel_filename = "Metrices_all_algorithms_Ro_E9_CCW.xlsx"
criteria_sheet_name = "DE"

# Select only the first 20 rows
criteria_df = criteria.iloc[:20].copy()

# Ensure "Run" column is present in criteria_df
if "Run" not in criteria_df.columns:
    criteria_df.insert(0, "Run", range(1, len(criteria_df) + 1))

# Add the "Fitness" column from results_df (first 20 rows)
criteria_df["Fitness"] = results_df.loc[:19, "Fitness"].values

# Select the required columns
export_columns_criteria = ["Run", "RMSE", "R2", "PBIAS", "NSE", "Fitness"]

# Check if the Excel file exists and append or create a new file
try:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a") as writer:
        criteria_df[export_columns_criteria].to_excel(writer, sheet_name=criteria_sheet_name, index=False)
except FileNotFoundError:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="w") as writer:
        criteria_df[export_columns_criteria].to_excel(writer, sheet_name=criteria_sheet_name, index=False)

print(f"Criteria successfully saved to {excel_filename} in sheet {criteria_sheet_name}.")